In [1]:
# if there are problems with imports run:
#!pip install ipynb
#!pip install import-ipynb
#!pip install ase
#!pip install git+https://github.com/henriasv/lammps-logfile.git
#!pip install scipy
#!pip install PySimpleGUI
#!pip install ipympl
import import_ipynb
import methods as mds
import plotting as plg
import gui

import lammps_logfile
import scipy
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2Tk
import PySimpleGUI as sg
import numpy as np
import math
import PySimpleGUI as sg
import matplotlib.widgets as wg
import matplotlib
import matplotlib.pyplot as plt

sg.theme("DarkBlue7")

importing Jupyter notebook from methods.ipynb
importing Jupyter notebook from plotting.ipynb
importing Jupyter notebook from gui.ipynb


'DarkBlue7'

In [ ]:


# a dictionary containing tuples (log, unit_types), keys are files' absolute paths
log_unit_types_dict = {}
all_files = []  # absolute paths to loaded files
tab_names = []
analyze_windows = {}  # dictionary of analyze windows
fit_cursor_dict = {}
rdf_cursor_dict = {}
view_cursor_dict = {}

main_window = gui.make_main_window()
main_window.maximize()
main_window.bring_to_front()

color = main_window.BackgroundColor

while True:
    win, event, values = sg.read_all_windows()
    # print(event, values)

    # Handles closing windows.
    if event == sg.WIN_CLOSED:
        if win in analyze_windows.values():
            win.close()
            key = list(analyze_windows.keys())[
                list(analyze_windows.values()).index(win)]
            analyze_windows[key] = None
            # clear all the data
            if key in list(mds.rdf_dict.keys()):
                del mds.rdf_dict[key]
            if key in list(mds.view_dict.keys()):
                del mds.view_dict[key]
            
        elif win == main_window:
            win.close()
            for window in analyze_windows.values():
                if window:
                    window.close()
            break

    # Handles 'Load Log Files...' file browser by loading all selected files.
    elif event == 'file_main':
        file_names = values['file_main'].split(';')
        for file_name in file_names:
            with open(file_name, 'r') as f:
                log, unit_types = mds.read_log_file(f)

                # Check if data was loaded properly.
                if log.get_keywords() == [] and unit_types == []:
                    sg.popup_ok(
                        f'An error occurred while attempting to load\n{file_name}.', title='Error')

                else:
                    # Handle case in which one of the selected files was alredy loaded
                    if file_name in all_files:
                        main_window[file_name].update(visible=True)
                        log_unit_types_dict.update(
                            {file_name: (log, unit_types)})
                        mds.update_combos_main(log.get_keywords(), len(
                            unit_types), main_window, file_name)
                        main_window.write_event_value(
                            f'plot_main_{file_name}', None)
                        main_window[file_name].select()

                    # Handle case in which a new file is loaded
                    elif file_name not in all_files:
                        all_files.append(file_name)
                        tab_name = file_name.split('/')[-1]
                        tab_names.append(tab_name)
                        log_unit_types_dict[file_name] = (log, unit_types)
                        tab = gui.make_new_tab_main(tab_name, file_name)
                        main_window['tab_group_main'].add_tab(tab)
                        main_window[f'fig_cv_{file_name}'].update(
                            background_color=color)
                        mds.update_combos_main(log.get_keywords(), len(
                            unit_types), main_window, file_name)
                        main_window.write_event_value(
                            f'plot_main_{file_name}', None)
                        main_window[file_name].select()
                        analyze_windows[file_name] = None

    # Handles 'Plot' buttons from all of main window's tabs
    elif 'plot_main' in event:
        # retrieve name of the log file
        tab_id = event.replace('plot_main_', '')
        # get data of a given file from the dictionary
        log, unit_types = log_unit_types_dict[tab_id]
        plg.plot_main(log, unit_types, main_window[f'fig_cv_{tab_id}'].TKCanvas,
                      main_window[f'controls_cv_{tab_id}'].TKCanvas, values, tab_id)

    # Opens analyze windows
    elif 'analyze' in event and not analyze_windows[event.replace('analyze_', '')]:
        win_id = event.replace('analyze_', '')  # retrieve name of the log file
        win_name = win_id.split('/')[-1]
        analyze_windows[win_id] = gui.make_analyze_window(win_name, win_id)
        log, unit_types = log_unit_types_dict[win_id]
        combo_xy = log.get_keywords()
        mds.update_combos_analyze(
            analyze_windows[win_id], values, win_id, combo_xy, len(unit_types))
        analyze_windows[win_id][f'fig_cv_rdf_{win_id}'].update(
            background_color=color)
        analyze_windows[win_id][f'fig_cv_fit_{win_id}'].update(
            background_color=color)
        analyze_windows[win_id].write_event_value(
            f'plot_rdf_{win_id}', None)
        analyze_windows[win_id].write_event_value(
            f'plot_fit_{win_id}', None)
        analyze_windows[win_id].write_event_value(
            f'plot_view_{win_id}', None)

    # Handles 'Load RDF files...' files browser
    elif 'file_rdf' in event:
        # retrieve name of the log file
        win_id = event.replace('file_rdf_', '')
        file_names = values[event].split(';')
        for file_name in file_names:
            with open(file_name, 'r') as f:
                mds.rdf_dict, rdf_cols_number = mds.read_RDF_file(f, win_id)

        combo_values = [i for i in range(1, rdf_cols_number-1)]
        analyze_windows[win_id][f'combo_colrdf_{win_id}'].update(
            disabled=False, values=combo_values, value=combo_values[0])
        analyze_windows[win_id][f'clear_rdf_data_{win_id}'].update(
            disabled=False)

    elif 'file_view' in event:
        # retrieve name of the log file
        win_id = event.replace('file_view_', '')
        file_names = values[event].split(';')
        for file_name in file_names:
            with open(file_name, 'r') as f:
                mds.view_dict = mds.read_dump_file(f, win_id)
        analyze_windows[win_id][f'clear_dump_data_{win_id}'].update(
            disabled=False)

    elif '+CLICKED+' in event:
        win_id = event[0].replace('table_fit_', '')
        selected_rows = plg.selected_rows_dict[win_id]
        clicked_row = event[2][0]
        if clicked_row in selected_rows:
            selected_rows.remove(clicked_row)
            new_selected_rows = selected_rows
        elif len(selected_rows) > 0:
            new_selected_rows = [selected_rows[-1], clicked_row]
        else:
            new_selected_rows = [clicked_row]
        plg.selected_rows_dict[win_id] = new_selected_rows

        win[f'table_fit_{win_id}'].update(select_rows=new_selected_rows)

    elif 'intersection' in event:
        win_id = event.replace('intersection_', '')
        xq = values[f'combo_xfit_{win_id}']
        n = values[f'combo_nfit_{win_id}'] - 1
        yq = values[f'combo_yfit_{win_id}']
        log, unit_types = log_unit_types_dict[win_id]
        x = log.get(xq, n)
        y = log.get(yq, n)
        u_x = mds.units(unit_types[n], xq)
        selected_rows = plg.selected_rows_dict[win_id]
        np.seterr(all='warn')
        if len(selected_rows) == 2:
            row1, row2 = selected_rows
            slope1, intercept1 = plg.plotting_fit_data[win_id]['table_rows'][row1][1:3]
            slope2, intercept2 = plg.plotting_fit_data[win_id]['table_rows'][row2][1:3]
            x_i, y_i = mds.intersection(slope1, intercept1, slope2, intercept2)
            if math.isnan(x_i):
                sg.popup_ok(
                    f'Error occurred while calculating intersection point. Lines are almost parallel.', title='Warning')
            analyze_windows[win_id][f'intersection_value_{win_id}'].update(
                value=f'{mds.format_number(x_i)} {u_x}')
            intersection_line = plg.plotting_fit_data[win_id]['intersection_line']
            if intersection_line:
                line = intersection_line.pop(0)
                line.remove()
                plg.plotting_fit_data[win_id]['intersection_line'] = None
                plg.plotting_fit_data[win_id]['fit_fig'].canvas.draw()
            if x_i <= x.max() and x_i >= x.min():
                ax_fit = plg.plotting_fit_data[win_id]['fit_axs']
                plg.plotting_fit_data[win_id]['intersection_line'] = ax_fit.plot(
                    [x_i, x_i], [y.min(), y.max()], linestyle='dashed', color='red')
                plg.plotting_fit_data[win_id]['fit_fig'].canvas.draw()
        else:
            sg.popup_ok(
                f'Before calculating intersection point make sure two lines from the table are selected.', title='Warning')

    elif 'plot_fit' in event:
        win_id = event.replace('plot_fit_', '')
        fig_cv = analyze_windows[win_id][f'fig_cv_fit_{win_id}'].TKCanvas
        controls_cv = analyze_windows[win_id][f'controls_cv_fit_{win_id}'].TKCanvas
        log, unit_types = log_unit_types_dict[win_id]
        axs = plg.plot_fit(log, unit_types, fig_cv, controls_cv,
                           values, win_id, analyze_windows[win_id])
        fit_cursor_dict[win_id] = wg.Cursor(axs, useblit=True, color='red',
                                            horizOn=False)

    # Handles 'Plot' buttons in analyze windows.
    elif 'plot_rdf' in event:
        win_id = event.replace('plot_rdf_', '')
        fig_cv = analyze_windows[win_id][f'fig_cv_rdf_{win_id}'].TKCanvas
        controls_cv = analyze_windows[win_id][f'controls_cv_rdf_{win_id}'].TKCanvas
        log, unit_types = log_unit_types_dict[win_id]
        axs = plg.plot_rdf(log, unit_types, fig_cv, controls_cv,
                           values, win_id, analyze_windows[win_id])
        rdf_cursor_dict[win_id] = (wg.Cursor(
            axs[0], useblit=True, color='red'), wg.Cursor(axs[1], useblit=True, color='red'))

    elif 'plot_view' in event:
        win_id = event.replace('plot_view_', '')
        fig_cv = analyze_windows[win_id][f'fig_cv_view_{win_id}'].TKCanvas
        controls_cv = analyze_windows[win_id][f'controls_cv_view_{win_id}'].TKCanvas
        log, unit_types = log_unit_types_dict[win_id]
        axs = plg.plot_view(log, unit_types, fig_cv, controls_cv,
                            values, win_id, analyze_windows[win_id])
        view_cursor_dict[win_id] = wg.Cursor(axs[0], useblit=True, color='red')

    elif 'clear_rdf_data' in event:
        win_id = event.replace('clear_rdf_data_', '')
        del mds.rdf_dict[win_id]
        analyze_windows[win_id][f'combo_colrdf_{win_id}'].update(
            disabled=True, values=[])
        analyze_windows[win_id][f'clear_rdf_data_{win_id}'].update(
            disabled=True)
        
    elif 'clear_dump_data' in event:
        win_id = event.replace('clear_dump_data_', '')
        del mds.view_dict[win_id]
        analyze_windows[win_id][f'clear_dump_data_{win_id}'].update(
            disabled=True)

    elif 'close_tab' in event:
        win_id = event.replace('close_tab_', '')
        main_window[win_id].update(visible=False)
        del log_unit_types_dict[win_id]

main_window.close()